This notebook should be run every time the PLPs selection process changes.

Here we filter samples variants so, that only PLPs are left. This is done to make it easier to collect dataset for the regression analysis.

In [ ]:
from ukbb_recessive.data_collection.variants import VariantFeatures
import glob
import os
import datetime
import multiprocessing as mp
import pandas as pd

# Original PLPs

In [ ]:
# original files with samples variants generated in RAP per chromosome
rap_files = glob.glob(".../450k/RAP_output_per_chr/*.all_parts_final.csv")

## file, that contains all PLPs found in the cohort
all_plps_file = (
    ".../450k/plp_selection/basic/new_gene_names/new_freq/new_relatedness/"
    "all_chr_total_presumable_plps_HFE_final_sorted.txt"
)

# output folder for PLPs
## bugfix with gnomad frequency -- now in %
output_folder= '.../450k/RAP_output_per_chr/filtered_plps/basic/new_gene_names/new_freq/new_relatedness'


# create output folder if not exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [ ]:
# filter out non-PLP variants for every chromosome

VariantFeatures().filter_plps_in_samples(
    rap_files = rap_files, 
    output_folder = output_folder, 
    all_plps_file = all_plps_file
)

In [ ]:
print ("Filtered PLPs files:\n")

! ls -lah $output_folder

# Numbers check for the paper

In [ ]:
filter_homozygous=True

cohort_plp_files = glob.glob(f"{output_folder}/chr*")

cohort_plps = VariantFeatures().read_sample_plps(cohort_plp_files, filter_homozygous=filter_homozygous).drop(['raw pos', 'raw ref', 'raw alt', 'alleles', 'GT.alleles'], axis=1)

In [ ]:
print (f"Number of PLPs in the cohort:", cohort_plps.shape[0])
print (f"Number of unique PLPs:", cohort_plps[['chrom', 'pos', 'ref', 'alt']].drop_duplicates().shape[0])

In [ ]:
homozygous = cohort_plps[cohort_plps['GT'] == '1/1']

print (f"Number of hom PLPs in the cohort:", homozygous.shape[0])
print (f"Number of unique hom PLPs:", homozygous[['chrom', 'pos', 'ref', 'alt']].drop_duplicates().shape[0])

In [ ]:
all_plps = pd.read_csv(all_plps_file, sep='\t')

all_plps